In [10]:
# Import libraries
import pandas as pd # needed for most operation
import numpy as np # needed for some array operations
from matplotlib import pyplot as plt #used for plots
import seaborn as sns
import datawig
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from importlib import reload

In [106]:
# Fetch Data
data = pd.read_csv('../data/RA_data.csv')
data.describe()
data.head()

,Treatment,Fever,Duration_of_pain,Sick_leave,Earlier_hospitalization,Workoverload,Familiy_history,Depression,Extremely_nervous,Stress,...,Failure_symptoms,Incoordination,neck_pain_intensity,low_back_pain_intensity,arm_left_pain_intensity,arm_right_pain_intensity,leg_left_pain_intensity,leg_right_pain_intensity,working_ability,Paidwork
0,1,1.0,10.0,0,1,NaN,0,0,0.0,0,...,1,NaN,0,8,8,7,6,6,NaN,0
1,3,1.0,10.0,0,1,0.0,1,0,0.0,1,...,1,1.0,7,8,7,4,0,0,7.0,1
2,1,1.0,10.0,0,1,NaN,0,0,6.0,1,...,1,0.0,3,5,0,2,0,3,NaN,0
3,5,1.0,10.0,0,1,NaN,0,0,1.0,1,...,0,NaN,2,9,0,0,9,5,NaN,0
4,1,1.0,10.0,0,1,NaN,0,0,0.0,1,...,1,NaN,0,7,5,0,0,0,NaN,0


In [107]:
# Drop columns
finalData = data.copy()
finalData.drop(labels='Workoverload',axis=1, inplace=True)
finalData.drop(labels='Relationship_with_colleagues',axis=1, inplace=True)
finalData.drop(labels='Trauma',axis=1, inplace=True)
finalData.drop(labels='working_ability',axis=1, inplace=True)

In [108]:
# Remove rows that have Treatment 4
finalData = finalData[finalData['Treatment'] != 4]

In [109]:
# Turn age into numerical value
ageMap = {'0-19': 1, '20-29': 2, '30-39': 3, '40-49': 4, '50-59': 5, '60-69': 6, '70-79': 7, '>=80': 8}
finalData['Age'] = finalData['Age'].map(ageMap)

In [69]:
dfCount = finalData.describe().loc['count']
numRows = finalData.shape[0]

outputs = dfCount.loc[dfCount.values < numRows]
outputs = list(outputs.index)

inputs = dfCount.loc[dfCount.values == numRows]
inputs = list(inputs.index)

In [110]:
finalData = finalData.fillna('')
finalData[outputs[:]] = finalData[outputs[:]].astype(str)

In [115]:
df_train, df_test = train_test_split(finalData, test_size=0.1, random_state=0)
imputedData = finalData
for output in outputs:
    imputer = datawig.SimpleImputer(
        input_columns= inputs,
        output_column= output,
        output_path= "./imputerModels/simpleModel_"+output,
    )

    imputer.fit(train_df=df_train)

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
[13:45:44] ../src/executor/graph_executor.cc:1995: Subgraph backend MKLDNN is activated.
2022-01-24 13:45:44,366 [INFO]  
========== start: fit model
2022-01-24 13:45:44,385 [WARNING]  Already bound, ignoring bind()
2022-01-24 13:45:44,620 [INFO]  Epoch[0] Batch [0-38]	Speed: 2740.82 samples/sec	cross-entropy=0.359860	Fever-accuracy=0.844551
2022-01-24 13:45:44,846 [INFO]  Epoch[0] Train-cross-entropy=0.161160
2022-01-24 13:45:44,850 [INFO]  Epoch[0] Train-Fever-accuracy=0.947635
2022-01-24 13:45:44,852 [INFO]  Epoch[0] Time cost=0.463
2022-01-24 13:45:44,860 [INFO]  Saved checkpoint to "./imputerModels/simpleModel_

In [ ]:
# Fill a new data set with the imputed models
imputedData = finalData.copy()
for output in outputs:
    outputModel = datawig.SimpleImputer.load("./imputerModels/simpleModel_"+output)
    predictions = outputModel.predict(finalData)
    imputedData.loc[imputedData[output] == "",output] = predictions.loc[predictions[output] == "",output+"_imputed"]

imputedData

In [ ]:
predictions = imputer.predict(df_test)
#Calculate f1 score
f1 = metrics.f1_score(predictions[outputs[0]], predictions[outputs[0]+'_imputed'])
print(f1)

#Print overall classification report
print(metrics.classification_report(predictions[outputs[0]], predictions[outputs[0]+'_imputed']))

In [ ]:
#finalDataImputed.describe()

In [ ]:
imputedData.to_pickle('../data/imputedData_1')

In [80]:
predictions = imputer.predict(df_test)
predictions

,Treatment,Fever,Duration_of_pain,Sick_leave,Earlier_hospitalization,Familiy_history,Depression,Extremely_nervous,Stress,Irrational_thoughts_risk_lasting,...,Incoordination,neck_pain_intensity,low_back_pain_intensity,arm_left_pain_intensity,arm_right_pain_intensity,leg_left_pain_intensity,leg_right_pain_intensity,Paidwork,Fever_imputed,Fever_imputed_proba
484,5,0.0,10.0,1,1,0,1,8.0,1,6.0,...,0.0,0,4,0,0,0,0,1,0.0,0.999994
1067,1,0.0,10.0,1,0,0,0,2.0,1,3.0,...,1.0,0,7,0,0,0,9,0,0.0,0.999993
1378,1,0.0,10.0,0,0,0,0,0.0,1,8.0,...,0.0,5,6,0,0,0,0,1,0.0,0.999928
302,2,1.0,10.0,0,1,0,0,1.0,0,9.0,...,1.0,9,6,3,3,0,0,0,1.0,0.999666
657,1,0.0,10.0,1,0,0,0,2.0,0,8.0,...,1.0,0,7,0,0,0,0,0,0.0,0.999993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,1,0.0,10.0,0,1,1,0,0.0,0,9.0,...,0.0,0,6,0,8,9,6,0,0.0,0.999996
188,5,1.0,7.0,0,1,0,0,0.0,1,5.0,...,1.0,0,9,0,0,6,6,0,1.0,0.997248
1044,2,0.0,10.0,0,1,1,0,3.0,1,5.0,...,NaN,3,4,0,0,0,4,1,0.0,0.989612
1431,5,0.0,9.0,0,1,1,0,4.0,1,9.0,...,1.0,0,8,0,0,5,5,0,0.0,0.999936
